In [427]:
import json
import pandas as pd
import numpy as np
from pandasql import sqldf
from pandas import json_normalize

In [428]:
df = pd.read_csv('data.csv', parse_dates=['application_date'], date_format='mixed')
df["contracts"] = df["contracts"].apply(lambda x: json.loads(x) if pd.notna(x) and x != '' else [])
df["contracts"] = df["contracts"].apply(lambda x: [x] if type(x) == dict else x)
df["id"] = df["id"].apply(lambda x: np.int64(x))

In [429]:
df_flat = df.explode('contracts')
df_flat.to_csv('df_flat.csv',header=True, index=False)

In [430]:
df_flat_normalized = json_normalize(df_flat['contracts'])
df_flat_normalized.to_csv('df_flat_normalized.csv',header=True, index=False)

In [431]:
final_df = df_flat.drop(columns=['contracts']).join(df_flat_normalized)
final_df['claim_date'] = pd.to_datetime(final_df['claim_date'], dayfirst=True).dt.date
final_df['contract_date'] = pd.to_datetime(final_df['contract_date'], dayfirst=True).dt.date

final_df.to_csv('final_df.csv',header=True, index=False)

In [447]:
query = """

SELECT
    count(*) as tot_claim_cnt_l180d
FROM
    final_df
WHERE claim_date >= DATE('now', '-200 days') and claim_date is not null
order by claim_date desc
"""

query = """
SELECT
    bank, sum(loan_summa)
FROM
    final_df
where bank not in ('LIZ', 'LOM', 'MKO', 'SUG') and bank is not null
and contract_date is not null
group by bank
"""

query = """
SELECT
    id, max(contract_date) as max_contract_date
FROM
    final_df
where summa is not null
group by id
having max_contract_date is not null
order by max_contract_date desc
"""


res = sqldf(query)
res
# res.to_csv('res.csv',header=True, index=False)

,id,max_contract_date
0,2926146,2024-01-06
1,2926090,2024-01-06
2,2926034,2024-01-06
3,2925978,2024-01-06
4,2926145,2024-01-05
...,...,...
85,2925211,2020-02-17
86,2926196,2019-10-30
87,2926195,2019-10-30
88,2926194,2019-04-05
